In [1]:
import os
%pwd

'/home/towet/Desktop/OpenProjects/MRI-CT-Translator/research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/towet/Desktop/OpenProjects/MRI-CT-Translator'

In [5]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir : Path 
    model_path : Path 
    dataset : Path 
    img_shape : tuple
    epochs : int
    lr : float
    l1 : str
    l2 : str
    
    

In [18]:
from MRICTTranslator.constants import * 
from MRICTTranslator.utils.common import read_yaml, create_directories

In [29]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
    def get_training_config(self) -> ModelTrainingConfig:
        config = self.config.training 
        create_directories([config.root_dir])
        params = self.params 
        dataset = 'artifacts/data_ingestion/dataset/Dataset/images/'
        training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            dataset=str(dataset),
            img_shape=params.IMAGE_SHAPE,
            epochs=params.EPOCHS,
            lr=params.LR,
            l1=params.L1,
            l2=params.L2,
            
            
        )
        return training_config

In [30]:
# mritoct
from os import listdir
from numpy import asarray
from numpy import vstack
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from matplotlib import pyplot as plt
import numpy as np
from datetime import datetime
from MRICTTranslator.utils.common import preprocess_data,train
from MRICTTranslator.models.cycle_gan import *

In [31]:
class Training:
	def __init__(self, config: ModelTrainingConfig):
		self.config = config
		
	# load all images in a directory into memory
	def load_images(self, path, size=(256,256)):
		data_list = list()
		# enumerate filenames in directory, assume all are images
		for filename in listdir(path):
			# load and resize the image
			pixels = load_img(path + filename, target_size=size)
			# convert to numpy array
			pixels = img_to_array(pixels)
			# store
			data_list.append(pixels)
		return asarray(data_list)
	def model_train(self):
		dataA = self.load_images(self.config.dataset + 'trainA/')
		dataB = self.load_images(self.config.dataset + 'trainB/')
		# load image data
		data = [dataA, dataB]
		dataset = preprocess_data(data)
		# define input shape based on the loaded dataset

		image_shape = dataset[0].shape[1:]
		# generator: A -> B
		g_model_AtoB = define_generator(image_shape)
		# generator: B -> A
		g_model_BtoA = define_generator(image_shape)
		# discriminator: A -> [real/fake]
		d_model_A = define_discriminator(image_shape)
		# discriminator: B -> [real/fake]
		d_model_B = define_discriminator(image_shape)
		# composite: A -> B -> [real/fake, A]
		c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
		# composite: B -> A -> [real/fake, B]
		c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)
		start1 = datetime.now()
		# train models
		train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset, epochs=self.config.epochs)

		stop1 = datetime.now()
		#Execution time of the model
		execution_time = stop1-start1
		print("Execution time is: ", execution_time)

    

In [32]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_training_config()
    model_train = Training(config=model_training_config)
    model_train.model_train()
except Exception as e:
    raise e
    

[2024-06-06 11:23:29,334: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-06 11:23:29,338: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-06 11:23:29,340: INFO: common: created directory at: artifacts]
[2024-06-06 11:23:29,343: INFO: common: created directory at: artifacts/training]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/dataset/images/trainA/'